In [4]:
import pymc3 as pm
import pandas as pd
import numpy as np
# Dependency to be removed
from poibin import PoiBin

ModuleNotFoundError: No module named 'poibin'

In [ ]:
# This is something I have readily available from my code.
# It will be much simpler for actual workshops
def get_goal_ps(ps):
    pb = PoiBin(ps)
    goals = []
    probs = []
    g = 0
    p = 1
    while ((p > 0.01) or (g <= 3)) and (g <= ps.shape[0]):
        p = pb.pmf(g)
        goals.append(g)
        probs.append(p)
        g += 1
    probs = probs[:-1]
    goals = goals[:-1]
    return probs, goals

dt = pd.read_csv("https://git.io/fNmRy")
dt = dt.loc[dt.season == 2017]
# For each match, collect the expected goals for each team
match_id, time, home_team, i_home, away_team, i_away, goal_home, goal_away, weight = [], [], [], [], [], [], [], [], []
# Create a parallel data with only results
match_id_only, time_only, home_team_only, i_home_only, away_team_only, i_away_only, goal_home_only, goal_away_only = [], [], [], [], [], [], [], []
# Create a dictionary of teams for the i_home/i_away column
teams = dt.home.unique()
team_dic = {team:i for i, team in enumerate(teams)}


matches = dt.groupby('match_id')
# Construct the weighted and non-weighted data
for i, match in enumerate(matches):
    m_id = match[0]
    # print(m_id)
    match = match[1]
    # Get the time in unix timestamp
    time_ = pend.parse(match.date.iloc[0])
    time_ = time_.int_timestamp
    ps = match.loc[match.side=='h', 'xg'].to_numpy()
    p_home, g_home = get_goal_ps(ps)
    ps = match.loc[match.side=='a', 'xg'].to_numpy()
    p_away, g_away = get_goal_ps(ps)
    for (g_h, p_h), (g_a, p_a) in product(zip(g_home, p_home), zip(g_away, p_away)):
        match_id.append(m_id)
        goal_home.append(g_h)
        goal_away.append(g_a)
        # We should have probabilities high enough for this not to be a problem
        weight.append(p_a * p_h)
        home_team.append(match.iloc[0, 2])
        i_home.append(team_dic[match.iloc[0, 2]])
        away_team.append(match.iloc[0, 3])
        i_away.append(team_dic[match.iloc[0, 3]])
        time.append(time_)
    match_id_only.append(m_id)
    time_only.append(time_)
    home_team_only.append(match.iloc[0, 2])
    away_team_only.append(match.iloc[0, 3])
    i_home_only.append(team_dic[match.iloc[0, 2]])
    i_away_only.append(team_dic[match.iloc[0, 3]])
    goal_home_only.append(match.iloc[0,4])
    goal_away_only.append(match.iloc[0,5])

dt_simple = pd.DataFrame.from_dict({'match_id':match_id_only, 'time':time_only, 'home_team':home_team_only,
                                    'away_team':away_team_only, 'goal_home': goal_home_only, 'goal_away':goal_away_only,
                                    'i_home':i_home_only, 'i_away': i_away_only})

